In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import RandomOverSampler,SMOTEN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from keras.models import Model
import kerastuner as kt
from keras.applications.mobilenet import preprocess_input
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense
from keras.layers.merge import concatenate

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**We will import the datasets from fer2013.csv dataset from Kaggle.**

In [ ]:
expression=pd.read_csv('../input/facialexpressionrecognition/fer2013.csv')
expression

In [ ]:
expression.emotion.unique()

**Each pixel row has 2304 values which can be broken down into 48*48 images**

In [ ]:
len(expression['pixels'][0].split())

**The data is distributed across training,testing and validation sets. Thus, we will seperate that data.**

In [ ]:
emot_training=expression.loc[expression['Usage']=='Training']
emot_test=expression.loc[expression['Usage']=='PrivateTest']
emot_valid=expression.loc[expression['Usage']=='PublicTest']

**We will now look at the data different distribution of data for test,validation and training data for each category of image.**

In [ ]:
pd.concat([emot_training['emotion'].value_counts(),emot_valid['emotion'].value_counts(),emot_test['emotion'].value_counts()],axis=1,keys=['Training','Validation','Testing'])

**Since the data for each category is not evenly distributed, we will do some random sampling to increase the size of data for certain caetgories using imblearn library.**

In [ ]:
dup_emo=emot_training['emotion']
dup_train=emot_training.drop('emotion',axis=1)

dup_emo_valid=emot_valid['emotion']
dup_valid=emot_valid.drop('emotion',axis=1)

sampling_dict_1={0:3995,1:1500,2:4097,3:7215,4:4830,5:3171,6:4965}
# sampling_dict_2={0:3995,1:1500,2:4097,3:4000,4:4000,5:3171,6:4000}
over_sample=SMOTEN(sampling_strategy=sampling_dict_1,random_state=101)
dup_train,dup_emo=over_sample.fit_resample(dup_train.values,dup_emo.values)

sampling_dict_2={0:467,1:200,2:496,3:895,4:653,5:415,6:607}
over_sample_2=SMOTEN(sampling_strategy=sampling_dict_2,random_state=101)
dup_valid,dup_emo_valid=over_sample_2.fit_resample(dup_valid.values,dup_emo_valid.values)
# under_sample=RandomUnderSampler(sampling_strategy=sampling_dict_2,random_state=101)
# steps=[('over',over_sample),('under',under_sample)]
# pipe=Pipeline(steps)
# dup,dup_emo=pipe.fit_resample(dup.values,dup_emo.values)

In [ ]:
emot_training=pd.DataFrame(dup_train,columns=['pixels','Usage'])
emot_training['emotion']=dup_emo

emot_valid=pd.DataFrame(dup_valid,columns=['pixels','Usage'])
emot_valid['emotion']=dup_emo_valid

In [ ]:
pd.concat([emot_training['emotion'].value_counts(),emot_valid['emotion'].value_counts()],axis=1)

In [ ]:
plt.figure(figsize=(12,6))
for each_image in range(1,10):
    temp=emot_training['pixels'][each_image].split()
    for i in range(len(temp)):
        temp[i]=int(temp[i])
    plt.subplot(3,3,each_image)
    plt.imshow(np.array(temp).reshape(48,48))

In [ ]:
train_emotions=emot_training['emotion']
train_emotions=keras.utils.to_categorical(train_emotions)

test_emotions=emot_test['emotion']
test_emotions=keras.utils.to_categorical(test_emotions)

valid_emotions=emot_valid['emotion']
valid_emotions=keras.utils.to_categorical(valid_emotions)


In [ ]:
train_images=np.uint8(emot_training['pixels'].str.split().tolist())
train_images=train_images.reshape((emot_training.shape[0],48,48,1))
train_images=train_images.astype('float32')/255

test_images=np.uint8(emot_test['pixels'].str.split().tolist())
test_images=test_images.reshape((emot_test.shape[0],48,48,1))
test_images=test_images.astype('float32')/255

valid_images=np.uint8(emot_valid['pixels'].str.split().tolist())
valid_images=valid_images.reshape((emot_valid.shape[0],48,48,1))
valid_images=valid_images.astype('float32')/255


In [ ]:
plt.imshow(test_images[0])

**We use stacked models for prediction of our data.
I have used 3 different neural models an stacked them up on another neural network.**

In [ ]:
model_1=keras.Sequential([
    keras.Input(train_images.shape[1:]),
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(64,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.BatchNormalization(renorm=True),
    layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.BatchNormalization(renorm=True),
    layers.Conv2D(256,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(256,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Flatten(),

    layers.Dense(units=192,activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(192,activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(7,activation='softmax')
])

opt=keras.optimizers.Adam(
    learning_rate=0.01,
#     beta_1=0.91
)  

model_1.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
    
    
model_2=keras.Sequential([
    keras.Input(train_images.shape[1:]),
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(64,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(64,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.BatchNormalization(renorm=True),
    layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.BatchNormalization(renorm=True),
    layers.Conv2D(256,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(256,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Flatten(),

    layers.Dense(units=128,activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(128,activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(7,activation='softmax')
])

opt=keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.91
)  

model_2.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)    


model_3=keras.Sequential([
    keras.Input(train_images.shape[1:]),
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(64,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(64,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.BatchNormalization(renorm=True),
    layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
#     layers.Conv2D(128,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.BatchNormalization(renorm=True),
    layers.Conv2D(256,kernel_size=3,activation='relu',padding='same'),
    layers.Conv2D(256,kernel_size=3,activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Flatten(),

    layers.Dense(units=32,activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(32,activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(7,activation='softmax')
])

opt=keras.optimizers.Adam(
    learning_rate=0.01,
    beta_1=0.95
)  

model_3.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
    
models=[model_1,model_2,model_3]
epochs_list=[25,25,25]

In [ ]:
# def fit_model(train_x,train_y,i):
#     model=models[i]
    
#     history=model.fit(train_x,train_y,epochs=epochs_list[i],verbose=0)
    
#     return model

# for i in range(3):
#     model=fit_model(train_images,train_emotions,i)
    
#     filename='model/model_' + str(i+4) + '.h5'
    
#     model.save(filename)
    
#     print('Saved' + filename)
    

In [ ]:
def load_all_models(n_models):
    all_models=[]
    for i in range(3):
        filename='../input/ensemble-models/model_' + str(i+4) + '.h5'
        model=load_model(filename)
        all_models.append(model)
        print(filename + 'loaded')
        
    return all_models

def stacked_model(members):
    for i in range(len(members)):
        model=members[i]
        for layer in model.layers:
            layer.trainable=False
            layer._name = 'ensemble_' + str(i+4) + '_' + layer.name
            
    ensemble_input=[model.input for model in members]
    ensemble_output=[model.output for model in members]

    merge=concatenate(ensemble_output)
    hidden=Dense(units=128,activation='relu')(merge)
    hidden=Dense(units=128,activation='relu')(hidden)
    hidden=Dense(units=128,activation='relu')(hidden)
    output=Dense(units=7,activation='softmax')(hidden)

    model=Model(inputs=ensemble_input,outputs=output)
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    
    return model

def fit_stacked_model(model,inputX,inputY):
    X=[inputX for _ in range(len(model.input))]
#     inputy_enc=to_categorical(inputY)
    
    model.fit(X,inputY,epochs=25,verbose=0)
    
def predict_stacked_model(model,inputx):
    X=[inputx for _ in range(len(model.input))]
    return model.predict(X,verbose=0)

members=load_all_models(3)
stacked_models=stacked_model(members)

fit_stacked_model(stacked_models,valid_images,valid_emotions)

yhat=predict_stacked_model(stacked_models,valid_images)
    

In [ ]:
stacked_models.input

In [ ]:
yhat2=np.argmax(yhat,axis=1)
acc=accuracy_score(np.argmax(valid_emotions,axis=1),yhat2)
acc

**We have a test accuracy of 64%+**

In [ ]:
yhat3=predict_stacked_model(stacked_models,test_images)
acc=accuracy_score(np.argmax(test_emotions,axis=1),np.argmax(yhat3,axis=1))
acc

In [ ]:
yhat4=predict_stacked_model(stacked_models,train_images)
print(classification_report(test_emotions,to_categorical(np.argmax(yhat3,axis=1))))
print(classification_report(train_emotions,to_categorical(np.argmax(yhat4,axis=1))))

In [ ]:
def map_emo(n):
    emotions_list = {0:'angry',1:'disgust',2:'fear',3:'happy',4:'sad',5:'surprise',6:'neutral'}
    return emotions_list[n]


def pred(image):
    inp=image.reshape(1,48,48,1)
    
    img=[inp for _ in stacked_models.input]
    
    print(map_emo(np.argmax(stacked_models.predict(img))))
    
    plt.imshow(image)

pred(train_images[3897])